# COVID19 Patient Data Analysis

### Aim: 
The aim of this analysis is to collate data coming from three different sources and further perform analysis on data
regarding COVID-19 patients using Pandas and MongoDB.

### Input
1. patient_data_1.csv
    Contains data about 843 patients.
2. patient_data_2.csv
    Contains data about 457 patients.
3. ‘patients’ collection
    Contains data about 11,502 patients.
    
### Output
1. Compute the current age of the patients
2. Obtain the ID’s of patients that were diagnosed with COVID-19
3. Create an aggregation pipeline to obtain COVID-19 patients who also had ‘Cough (finding)’ and ‘Fever
   (finding)’
4. Create a new aggregation pipeline to join patient data for 8,820 COVID-19 patients. Remove dead patients
   here
5. Based on the previous pipeline, create a new pipeline to calculate the total number of alive COVID-19
   patients for each gender

### Import libraries

In [1]:
%config IPCompleter.use_jedi = False
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
import datetime
import pprint
from bson.objectid import ObjectId

### Import the ‘patient_data_1.csv’ and ‘patient_data_2.csv’ into pandas DataFrames and harmonize them.

1. Import patient_data_1.csv file 

In [2]:
#Read csv file
patient_data_1=pd.read_csv('patient_data_1.csv')
patient_data_1.head(3)

,Id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P9497,1948-07-23,S,white,nonhispanic,Male,Framingham,Middlesex County,Massachusetts
1,P6183,1997-12-12,NaN,white,hispanic,Female,Springfield,Hampden County,Massachusetts
2,P4358,2000-05-10,NaN,white,nonhispanic,Female,Quincy,Norfolk County,Massachusetts


In [3]:
patient_data_1.shape

(843, 9)

Change Male and Female values in patient_data_1 ||
Change column name id to "_id"

In [5]:
#check missing values
patient_data_1.isnull().sum()

Id             0
BIRTHDATE      0
MARITAL      277
RACE           0
ETHNICITY      0
GENDER         0
CITY           0
COUNTY         0
STATE          0
dtype: int64

In [6]:
#Replace Female to F and Male to M
patient_data_1.loc[(patient_data_1.GENDER == 'Male'),"GENDER"]="M"
patient_data_1.loc[(patient_data_1.GENDER == 'Female'),"GENDER"]="F"

In [7]:
patient_df1=patient_data_1.rename(columns= {"Id":"_id"})
patient_df1.head(3)

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P9497,1948-07-23,S,white,nonhispanic,M,Framingham,Middlesex County,Massachusetts
1,P6183,1997-12-12,NaN,white,hispanic,F,Springfield,Hampden County,Massachusetts
2,P4358,2000-05-10,NaN,white,nonhispanic,F,Quincy,Norfolk County,Massachusetts


2. Import patient_data_2.csv file ||  convert column names to upper case

In [8]:
#Read csv file
patient_data_2=pd.read_csv('patient_data_2.csv')
patient_data_2.columns=patient_data_2.columns.str.upper()

Change non-hispanic to nonhispanic  || Change column name id to _id

In [9]:
patient_data_2.loc[(patient_data_2.ETHNICITY == 'non-hispanic'),"ETHNICITY"]="nonhispanic"
patient_df2=patient_data_2.rename(columns= {"ID":"_id"})
patient_df2.head(3)

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P294,1986-02-28,M,white,hispanic,M,North Brookfield,Worcester County,Massachusetts
1,P1286,1989-07-03,M,asian,nonhispanic,M,Winthrop,Suffolk County,Massachusetts
2,P6033,1968-01-13,M,black,nonhispanic,M,Marshfield,Plymouth County,Massachusetts


###  From the ‘subjects’ collection, retrieve the patients who are alive. Print the number of patients retrieved.

In [10]:
# connecting to database server
client = MongoClient('localhost', 27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [ ]:
# list databases in the database server
client.list_database_names()

In [16]:
#Selecting patientdata database
db=client.patientdata

#check the list of collections inside patientdata database
db.list_collection_names()

['conditions', 'patients']

In [17]:
#Rename collection for the analysis
mydb=client["patientdata"]
patients_coll=mydb["patients"]
conditions_coll=mydb["conditions"]
con_c=mydb.conditions

In [18]:
#Count documents in both collections
c_p=patients_coll.count_documents({})
c_c=conditions_coll.count_documents({})
print( f"Documents in patient collections {c_p}.")
print( f"Documents in condition collections {c_c}.")

Documents in patient collections 11052.
Documents in condition collections 54731.


In [19]:
#Check the one record of patient collection
patients_coll.find_one()

{'_id': 'P1',
 'BIRTHDATE': '2017-08-24',
 'DEATHDATE': nan,
 'MARITAL': nan,
 'RACE': 'white',
 'ETHNICITY': 'nonhispanic',
 'GENDER': 'M',
 'CITY': 'Springfield',
 'COUNTY': 'Hampden County',
 'STATE': 'Massachusetts'}

In [20]:
#From the patient collection, retrieve the patients who are alive. Print the number of patient retrieved.
pt_alive =patients_coll.count_documents({"DEATHDATE": float("nan")})
print(f"Number of alive patients: {pt_alive}.")

Number of alive patients: 8700.


In [21]:
#Store alive patients into dataframe
pt_list=list(patients_coll.find({"DEATHDATE": float("nan")}))
patients_d=pd.DataFrame(pt_list)

#Remove DEATHDATE column from dataframe
patient_df=patients_d.drop(columns=["DEATHDATE"], axis=1)
patient_df.head(3)

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P1,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
1,P10,2005-01-16,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
2,P100,1946-09-09,M,white,nonhispanic,M,Fall River,Bristol County,Massachusetts


###  Concatenate the above three DataFrames. You should get 10,000 patients. 

In [22]:
#Concatenate three dataframe patient_df1, patients_df2, and patient_df
con_df=pd.concat([patient_df,patient_df1,patient_df2])
con_df.head()

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P1,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
1,P10,2005-01-16,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
2,P100,1946-09-09,M,white,nonhispanic,M,Fall River,Bristol County,Massachusetts
3,P1000,1997-10-13,NaN,white,nonhispanic,F,Needham,Norfolk County,Massachusetts
4,P10000,1982-07-05,M,white,nonhispanic,F,Foxborough,Norfolk County,Massachusetts


## 1. Compute the current age of the patients and store them in another column called ‘AGE’. 

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
#Convert BIRTHDATE into datetiem format
con_df["BIRTHDATE"]=pd.to_datetime(con_df.BIRTHDATE)

#Calculate age using lambda function
con_df["age"]=con_df["BIRTHDATE"].apply(lambda x: (pd.datetime.now().year-x.year))
con_df.head()

<ipython-input-23-bc0bf20bece3>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  con_df["age"]=con_df["BIRTHDATE"].apply(lambda x: (pd.datetime.now().year-x.year))


,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE,age
0,P1,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts,4
1,P10,2005-01-16,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts,16
2,P100,1946-09-09,M,white,nonhispanic,M,Fall River,Bristol County,Massachusetts,75
3,P1000,1997-10-13,NaN,white,nonhispanic,F,Needham,Norfolk County,Massachusetts,24
4,P10000,1982-07-05,M,white,nonhispanic,F,Foxborough,Norfolk County,Massachusetts,39


###  Insert the above DataFrame to a collection named ‘patients_clean’. Display the number of documents in |the collection to make sure that all the records have been inserted properly

In [24]:
clean_df=con_df
clean_df

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE,age
0,P1,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts,4
1,P10,2005-01-16,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts,16
2,P100,1946-09-09,M,white,nonhispanic,M,Fall River,Bristol County,Massachusetts,75
3,P1000,1997-10-13,NaN,white,nonhispanic,F,Needham,Norfolk County,Massachusetts,24
4,P10000,1982-07-05,M,white,nonhispanic,F,Foxborough,Norfolk County,Massachusetts,39
...,...,...,...,...,...,...,...,...,...,...
452,P10416,2016-06-22,NaN,black,nonhispanic,M,Lynn,Essex County,Massachusetts,5
453,P2170,1956-01-21,M,white,hispanic,M,Westport,Bristol County,Massachusetts,65
454,P1082,1962-01-06,M,white,nonhispanic,M,Kingston,Plymouth County,Massachusetts,59
455,P6672,1971-12-02,S,white,nonhispanic,F,Webster,Worcester County,Massachusetts,50


In [25]:
#Create collection name "patient_clean"
collection=db["patients_clean"]
db.list_collection_names()

['conditions', 'patients']

In [26]:
#Insert dataframe to "patient_clean" collection
clean_df.reset_index(inplace=True)
clean_df_dict=clean_df.to_dict('records')
collection.insert_many(clean_df_dict)

In [27]:
#Count number of documents inside the "patients_clean" collection
pt_count=collection.count_documents({})
print(f"Number of patient in 'patients_clean'collection: {pt_count}.")

Number of patient in 'patients_clean'collection: 10000.


## 2. Obtain the ID’s of patients that were diagnosed with COVID-19. 

In [28]:
#Check record of conditions collection
conditions_coll.find_one()

{'_id': 'E1', 'patient': 'P1', 'conditions': ['Otitis media']}

In [29]:
#Count number of the patients who were diagnosed of COVID-19
pt_covid19=conditions_coll.count_documents({"conditions":"COVID-19"})
print(f"Number of patient diagnosed with COVID-19: {pt_covid19}.")

Number of patient diagnosed with COVID-19: 8820.


In [85]:
#Obtain ID's of the patients that were diagnosed of COVID-19
covid19_id=list(con_c.find ({"conditions":"COVID-19"},{"_id":1,"patient":1}))
len(covid19_id)

8820

In [42]:
covid_pt=list(con_c.find ({"conditions":"COVID-19"},{"_id":1,"patient":1}))

## 3. Create an aggregation pipeline to obtain COVID-19 patients who also had ‘Cough (finding)’ and ‘Fever|(finding)’. 

In [32]:
#count of the patients with COVID-19, cough, and fever
pt_mul_cond= conditions_coll.count_documents({'$and': [{"conditions":"COVID-19"},{"conditions":'Cough (finding)'},
                                      {"conditions":'Fever (finding)'}]})
print(f"Number of patient diagnosed with COVID-19 and also have cough and fever: {pt_mul_cond}.")

Number of patient diagnosed with COVID-19 and also have cough and fever: 5334.


In [84]:
# No of the patients with COVID-19, cough, and fever
covid19_multiple_symptom=list (conditions_coll.find ({'$and': [{"conditions":"COVID-19"},{"conditions":'Cough (finding)'},
                                      {"conditions":'Fever (finding)'}]}))
len(covid19_multiple_symptom)

5334

## 3. Create a new aggregation pipeline to join patient data for 8,820 COVID-19 patients. 

In [76]:
covid_pt_list=list(con_c.find ({"conditions":"COVID-19"},{"_id":1,"patient":1}))
covid_pt_df=pd.DataFrame(covid_pt_list)
covid_pt_df.head(2)

,_id,patient
0,E10,P4
1,E100,P26


In [78]:
#Rename column names
covid_pt_df2=covid_pt_df.rename(columns={"_id":"No_id"})
covid_pt_df3=covid_pt_df2.rename(columns={"patient":"_id"})
covid_pt_df3.head(2)

,No_id,_id
0,E10,P4
1,E100,P26


In [64]:
##Create new collection named "covid_coll" and insert newly formed dataframe
collection_covid=db["covid_coll"]
covid_pt_df3.reset_index(inplace=True)
covid_pt_df3_dict=covid_pt_df3.to_dict('records')
collection_covid.insert_many(covid_pt_df3_dict)

In [82]:
db.list_collection_names()

['covid_coll', 'conditions', 'patients', 'patients_clean']

In [83]:
#Use look up function and match function to join two collection
rrt=list(db.patients_clean.aggregate([
    {
        '$lookup':
        {
            'from':"covid_coll"  ,
            'localField':'_id'  ,
            'foreignField':'_id'  ,
            'as': 'hhhhh'  
        }},
    {
        "$match":{"hhhhh":{"$ne":[]}}
    }
        
    
]))

In [81]:
#Number of patients in the new aggreagation pipline
len(rrt)

8463

## 5. Based on the previous pipeline, create a new pipeline to calculate the total number of alive COVID-19 patients for each gender. Display the results. 

In [86]:
#Convert previous list to dataframe
gender_df=pd.DataFrame(rrt)
gender_df.head(2)

,_id,index,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE,age,hhhhh
0,P1,0,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts,4,"[{'_id': 'P1', 'index': 3573, 'No_id': 'E3'}]"
1,P100,2,1946-09-09,M,white,nonhispanic,M,Fall River,Bristol County,Massachusetts,75,"[{'_id': 'P100', 'index': 5398, 'No_id': 'E403'}]"


In [87]:
#Create new collection to calculate genders
collection_gender_covid=db["covid_gender"]
gender_df_dict=gender_df.to_dict('records')
collection_gender_covid.insert_many(gender_df_dict)

In [91]:
results_male = collection_gender_covid.aggregate([
    {
        '$match':{'GENDER':'M'}
    }
])

len(list(results_male))

3992

In [92]:
results_female = collection_gender_covid.aggregate([
    {
        '$match':{'GENDER':'F'}
    }
])

len(list(results_female))

4471